# 데이터 탐색 및 DB 연결 테스트

이 노트북은 가격 예측 프로젝트의 데이터 탐색과 데이터베이스 연결을 테스트합니다.

## 1. 라이브러리 임포트

In [1]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

## 2. 데이터베이스 연결 테스트

In [2]:
from src.data.db_connector import DatabaseConnector, SessionManager

# 데이터베이스 연결 테스트
print("=" * 50)
print("데이터베이스 연결 테스트")
print("=" * 50)

try:
    with DatabaseConnector() as db:
        print("✓ 데이터베이스 연결 성공!\n")
        
        # 테이블 목록 조회
        tables = db.get_table_list()
        print(f"✓ 사용 가능한 테이블 수: {len(tables)}")
        print("\n테이블 목록:")
        for i, table in enumerate(tables[:10], 1):
            print(f"  {i}. {table}")
        
        if len(tables) > 10:
            print(f"  ... 외 {len(tables) - 10}개")
        
        print("\n" + "=" * 50)
        print("연결 테스트 완료")
        print("=" * 50)
        
except Exception as e:
    print(f"✗ 연결 실패: {e}")

데이터베이스 연결 테스트


INFO:src.data.db_connector:Successfully connected to database: qai
INFO:src.data.db_connector:Database connection pool disposed


✓ 데이터베이스 연결 성공!

✓ 사용 가능한 테이블 수: 0

테이블 목록:

연결 테스트 완료


## 3. Step 1: 종목 필터링 (마스터 리스트 생성)

목표: NYSE, NASDAQ, AMEX 종목 중 조건을 만족하는 종목 추출

In [3]:
# ORM 모델 임포트
from src.data.models import (
    DS2CtryQtInfo, 
    DS2Exchange, 
    DS2PrimQtPrc,
    VwDs2Pricing,
    VwDs2MktCap
)
from sqlalchemy import func, and_, or_
from sqlalchemy.orm import aliased

print("✓ ORM 모델 임포트 완료")

✓ ORM 모델 임포트 완료


In [4]:
# Step 1: NYSE, NASDAQ, AMEX 종목 리스트 조회 (ORM 사용)
print("=" * 80)
print("Step 1: 종목 리스트 조회 (ORM)")
print("=" * 80)

try:
    with DatabaseConnector() as db:
        with SessionManager(db) as session:
            # 1. 먼저 미국 거래소 확인
            print("\n[1] 미국 거래소 조회...")
            us_exchanges = session.query(DS2Exchange).filter(
                DS2Exchange.ExchCtryCode == 'US'
            ).all()
            
            print(f"✓ 미국 거래소 수: {len(us_exchanges)}")
            for exch in us_exchanges[:5]:
                print(f"  - {exch.ExchName} ({exch.ExchMnem}): {exch.ExchIntCode}")
            
            # 2. NYSE, NASDAQ, AMEX 필터링
            print("\n[2] NYSE, NASDAQ, AMEX 필터링...")
            target_exchange_names = ['NYSE', 'NASDAQ', 'AMEX']
            target_exchanges = session.query(DS2Exchange).filter(
                and_(
                    DS2Exchange.ExchCtryCode == 'US',
                    or_(
                        DS2Exchange.ExchName.like('%NYSE%'),
                        DS2Exchange.ExchName.like('%NASDAQ%'),
                        DS2Exchange.ExchName.like('%AMEX%')
                    )
                )
            ).all()
            
            print(f"✓ 대상 거래소:")
            for exch in target_exchanges:
                print(f"  - {exch.ExchName} (코드: {exch.ExchIntCode})")
            
            target_exch_codes = [exch.ExchIntCode for exch in target_exchanges]
            
            # 3. 해당 거래소의 종목 조회
            print(f"\n[3] 종목 조회 (거래소 코드: {target_exch_codes})...")
            
            # DS2PrimQtPrc에서 해당 거래소의 InfoCode를 서브쿼리로 조회
            subquery = session.query(DS2PrimQtPrc.InfoCode).filter(
                DS2PrimQtPrc.ExchIntCode.in_(target_exch_codes)
            ).distinct().subquery()
            
            # DS2CtryQtInfo에서 종목 정보 조회
            stocks = session.query(
                DS2CtryQtInfo.InfoCode,
                DS2CtryQtInfo.DsCode,
                DS2CtryQtInfo.DsQtName,
                DS2CtryQtInfo.Region,
                DS2CtryQtInfo.IsPrimQt,
                DS2CtryQtInfo.StatusCode,
                DS2CtryQtInfo.DelistDate,
            ).filter(
                and_(
                    DS2CtryQtInfo.InfoCode.in_(subquery),
                    DS2CtryQtInfo.Region == 'US',
                    DS2CtryQtInfo.IsPrimQt == 1  # 주요 시세만
                )
            ).limit(100).all()  # 일단 100개만 샘플로
            
            print(f"✓ 조회된 종목 수: {len(stocks)} (샘플 100개)")
            
            # DataFrame으로 변환
            df_stocks = pd.DataFrame([
                {
                    'InfoCode': s.InfoCode,
                    'DsCode': s.DsCode,
                    'DsQtName': s.DsQtName,
                    'Region': s.Region,
                    'IsPrimQt': s.IsPrimQt,
                    'StatusCode': s.StatusCode,
                    'DelistDate': s.DelistDate,
                }
                for s in stocks
            ])
            
            print(f"\n✓ 데이터프레임 생성 완료:")
            print(f"  - Shape: {df_stocks.shape}")
            print(f"  - 상장폐지 종목: {df_stocks['DelistDate'].notna().sum()}")
            
            print(f"\n샘플 데이터 (처음 10개):")
            print(df_stocks.head(10))
            
except Exception as e:
    print(f"✗ 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    df_stocks = pd.DataFrame()

Step 1: 종목 리스트 조회 (ORM)


INFO:src.data.db_connector:Successfully connected to database: qai
/var/folders/tf/zjbsnpb54mscvn_67zr8r12w0000gn/T/ipykernel_44161/390371172.py:58: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  DS2CtryQtInfo.InfoCode.in_(subquery),



[1] 미국 거래소 조회...
✓ 미국 거래소 수: 51
  - NYSE American (ASE): 12
  - OTC Bulletin Bd. (XBQ): 27
  - Boston (BOS): 36
  - Cincinnati (CIN): 42
  - Midwest (MID): 124

[2] NYSE, NASDAQ, AMEX 필터링...
✓ 대상 거래소:
  - NYSE American (코드: 12)
  - Nasdaq (코드: 135)
  - Nasdaq Smallcap (코드: 140)
  - Nasdaq Interlisted (코드: 195)
  - NYSE ARCA (코드: 244)
  - AMEX (코드: 252)
  - NYSE (코드: 254)
  - NASDAQ (코드: 255)
  - BATS-NYSE/NYSE MKT (코드: 345)
  - BATS-NASDAQ MKT (코드: 346)
  - Nasdaq Basic (코드: 348)
  - NYSE BQT (코드: 355)
  - Boston SE for Nasdaq Capital Market (코드: 356)
  - Cboe EDGX for Nasdaq Capital Market (코드: 357)
  - Philadelphia for Nasdaq Capital Market (코드: 358)
  - NASDAQ Basic Canada (코드: 381)
  - Nasdaq Alt Dis Facility (코드: 395)
  - NYSE ARCA Book (코드: 407)
  - NYSE Texas (코드: 413)

[3] 종목 조회 (거래소 코드: [12, 135, 140, 195, 244, 252, 254, 255, 345, 346, 348, 355, 356, 357, 358, 381, 395, 407, 413])...
✓ 조회된 종목 수: 100 (샘플 100개)

✓ 데이터프레임 생성 완료:
  - Shape: (100, 7)
  - 상장폐지 종목: 60

샘플 데이터 (처음 10

INFO:src.data.db_connector:Database connection pool disposed


In [5]:
df_stocks

,InfoCode,DsCode,DsQtName,Region,IsPrimQt,StatusCode,DelistDate
0,5029,14746M,LAWSON SOFTWARE DEAD - 06/07/11,US,1,D,2011-07-06
1,4556,14666C,ONCOLYTICS BIOTECH,US,1,A,NaT
2,50310,992017,HELEN OF TROY,US,1,A,NaT
3,42992,873886,ALTABA DEAD - DELIST.12/10/19,US,1,D,2019-10-12
4,39979,719563,SUN MICROSYSTEMS DEAD - ACQD.BY 719618,US,1,D,2010-01-27
...,...,...,...,...,...,...,...
95,58923,923340,COMMERCE BCSH.,US,1,A,NaT
96,54655,519999,WESBANCO,US,1,A,NaT
97,58925,884327,CUBIST PHARMACEUTICALS DEAD - ACQD.905019,US,1,D,2015-01-22
98,71782,357985,ABERDEEN EMM.SM.CO.OPPS. FD. DEAD - DELIST.30/...,US,1,D,2018-04-30
